## Indic Languages Normalizer

This code is from: https://github.com/anoopkunchukuttan/indic_nlp_library

Also use Indic Numtowords: https://github.com/raj-sutariya/indic-num2words, https://github.com/AI4Bharat/indic-numtowords

This code has been modified by Kurian to suit to Whisper-normalizer style of coding and the logic for Malayalam normalization is expanded beyond the Indic NLP library by Dr Kavya.

In [ ]:
# | default_exp indic

In [ ]:
# | hide
from nbdev.showdoc import show_doc

In [ ]:
# | export
import re
from indic_numtowords import num2words


from whisper_normalizer import langinfo

In [ ]:
# | export

#  Copyright (c) 2013-present, Anoop Kunchukuttan
#  All rights reserved.
#
#  This source code is licensed under the MIT license found in the
#  LICENSE file in the root directory of this source tree.
#

# Program for normalization of text written in Unicode. This is mainly geared towards Indic scripts
#
# @author Anoop Kunchukuttan
#


class NormalizerI(object):
    """
    The normalizer classes do the following:
    * Some characters have multiple Unicode codepoints. The normalizer chooses a single standard representation
    * Some control characters are deleted
    * While typing using the Latin keyboard, certain typical mistakes occur which are corrected by the module
    Base class for normalizer. Performs some common normalization, which includes:
    * Byte order mark, word joiner, etc. removal
    * ZERO_WIDTH_NON_JOINER and ZERO_WIDTH_JOINER removal
    * ZERO_WIDTH_SPACE and NO_BREAK_SPACE replaced by spaces
    Script specific normalizers should derive from this class and override the normalize() method.
    They can call the super class 'normalize() method to avail of the common normalization
    """

    BYTE_ORDER_MARK = "\ufeff"
    BYTE_ORDER_MARK_2 = "\ufffe"
    WORD_JOINER = "\u2060"
    SOFT_HYPHEN = "\u00ad"

    ZERO_WIDTH_SPACE = "\u200b"
    NO_BREAK_SPACE = "\u00a0"

    ZERO_WIDTH_NON_JOINER = "\u200c"
    ZERO_WIDTH_JOINER = "\u200d"

    def _normalize_punctuations(self, text):
        """
        Normalize punctuations.
        Applied many of the punctuation normalizations that are part of MosesNormalizer
        from sacremoses
        """
        text = text.replace(NormalizerI.BYTE_ORDER_MARK, "")
        text = text.replace("„", r'"')
        text = text.replace("“", r'"')
        text = text.replace("”", r'"')
        text = text.replace("–", r"-")
        text = text.replace("—", r" - ")
        text = text.replace("´", r"'")
        text = text.replace("‘", r"'")
        text = text.replace("‚", r"'")
        text = text.replace("’", r"'")
        text = text.replace("''", r'"')
        text = text.replace("´´", r'"')
        text = text.replace("…", r"...")

        return text

    def normalize(self, text):
        pass

In [ ]:
# | export
class BaseNormalizer(NormalizerI):
    """Common class used in most of indic languages inherit from this code."""

    def __init__(
        self,
        lang,
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
    ):

        self.lang = lang
        self.remove_nuktas = remove_nuktas
        self.nasals_mode = nasals_mode
        self.do_normalize_chandras = do_normalize_chandras
        self.do_normalize_vowel_ending = do_normalize_vowel_ending

        self._init_normalize_chandras()
        self._init_normalize_nasals()
        self._init_normalize_vowel_ending()
        # self._init_visarga_correction()

    def _init_normalize_vowel_ending(self):

        if self.lang in langinfo.IE_LANGUAGES:
            self.fn_vowel_ending = self._normalize_word_vowel_ending_ie
        elif self.lang in langinfo.DRAVIDIAN_LANGUAGES:
            self.fn_vowel_ending = self._normalize_word_vowel_ending_dravidian
        else:
            self.fn_vowel_ending = lambda x: x

    def _init_normalize_chandras(self):

        substitution_offsets = [
            [0x0D, 0x0F],  # chandra e, independent
            [0x11, 0x13],  # chandra o, independent
            [0x45, 0x47],  # chandra e , 0xde],pendent
            [0x49, 0x4B],  # chandra o , 0xde],pendent
            # [0x72 , 0x0f], # mr: chandra e, independent
            [0x00, 0x02],  # chandrabindu
            [0x01, 0x02],  # chandrabindu
        ]

        self.chandra_substitutions = [
            (
                langinfo.offset_to_char(x[0], self.lang),
                langinfo.offset_to_char(x[1], self.lang),
            )
            for x in substitution_offsets
        ]

    def _normalize_chandras(self, text):
        for match, repl in self.chandra_substitutions:
            text = text.replace(match, repl)
        return text

    def _init_to_anusvaara_strict(self):
        """
        `r1_nasal=re.compile(r'\\u0919\\u094D([\\u0915-\\u0918])')`
        """

        pat_signatures = [
            [0x19, 0x15, 0x18],
            [0x1E, 0x1A, 0x1D],
            [0x23, 0x1F, 0x22],
            [0x28, 0x24, 0x27],
            [0x29, 0x24, 0x27],
            [0x2E, 0x2A, 0x2D],
        ]

        halant_offset = 0x4D
        anusvaara_offset = 0x02

        pats = []

        for pat_signature in pat_signatures:
            pat = re.compile(
                r"{nasal}{halant}([{start_r}-{end_r}])".format(
                    nasal=langinfo.offset_to_char(pat_signature[0], self.lang),
                    halant=langinfo.offset_to_char(halant_offset, self.lang),
                    start_r=langinfo.offset_to_char(pat_signature[1], self.lang),
                    end_r=langinfo.offset_to_char(pat_signature[2], self.lang),
                )
            )
            pats.append(pat)

        repl_string = "{anusvaara}\\1".format(
            anusvaara=langinfo.offset_to_char(anusvaara_offset, self.lang)
        )

        self.pats_repls = (pats, repl_string)

    def _to_anusvaara_strict(self, text):

        pats, repl_string = self.pats_repls
        for pat in pats:
            text = pat.sub(repl_string, text)

        return text

    def _init_to_anusvaara_relaxed(self):
        """
        `r1_nasal=re.compile(r'\\u0919\\u094D([\\u0915-\\u0918])')`
        """

        nasals_list = [0x19, 0x1E, 0x23, 0x28, 0x29, 0x2E]
        nasals_list_str = ",".join(
            [langinfo.offset_to_char(x, self.lang) for x in nasals_list]
        )

        halant_offset = 0x4D
        anusvaara_offset = 0x02

        pat = re.compile(
            r"[{nasals_list_str}]{halant}".format(
                nasals_list_str=nasals_list_str,
                halant=langinfo.offset_to_char(halant_offset, self.lang),
            )
        )

        repl_string = "{anusvaara}".format(
            anusvaara=langinfo.offset_to_char(anusvaara_offset, self.lang)
        )

        self.pats_repls = (pat, repl_string)

    def _to_anusvaara_relaxed(self, text):
        pat, repl_string = self.pats_repls
        return pat.sub(repl_string, text)

    def _init_to_nasal_consonants(self):
        """
        `r1_nasal=re.compile(r'\\u0919\\u094D([\\u0915-\\u0918])')`
        """

        pat_signatures = [
            [0x19, 0x15, 0x18],
            [0x1E, 0x1A, 0x1D],
            [0x23, 0x1F, 0x22],
            [0x28, 0x24, 0x27],
            [0x29, 0x24, 0x27],
            [0x2E, 0x2A, 0x2D],
        ]

        halant_offset = 0x4D
        anusvaara_offset = 0x02

        pats = []
        repl_strings = []

        for pat_signature in pat_signatures:
            pat = re.compile(
                r"{anusvaara}([{start_r}-{end_r}])".format(
                    anusvaara=langinfo.offset_to_char(anusvaara_offset, self.lang),
                    start_r=langinfo.offset_to_char(pat_signature[1], self.lang),
                    end_r=langinfo.offset_to_char(pat_signature[2], self.lang),
                )
            )
            pats.append(pat)
            repl_string = "{nasal}{halant}\\1".format(
                nasal=langinfo.offset_to_char(pat_signature[0], self.lang),
                halant=langinfo.offset_to_char(halant_offset, self.lang),
            )
            repl_strings.append(repl_string)

        self.pats_repls = list(zip(pats, repl_strings))

    def _to_nasal_consonants(self, text):

        for pat, repl in self.pats_repls:
            text = pat.sub(repl, text)

        return text

    def _init_normalize_nasals(self):

        if self.nasals_mode == "to_anusvaara_strict":
            self._init_to_anusvaara_strict()
        elif self.nasals_mode == "to_anusvaara_relaxed":
            self._init_to_anusvaara_relaxed()
        elif self.nasals_mode == "to_nasal_consonants":
            self._init_to_nasal_consonants()

    def _normalize_nasals(self, text):
        if self.nasals_mode == "to_anusvaara_strict":
            return self._to_anusvaara_strict(text)
        elif self.nasals_mode == "to_anusvaara_relaxed":
            return self._to_anusvaara_relaxed(text)
        elif self.nasals_mode == "to_nasal_consonants":
            return self._to_nasal_consonants(text)
        else:
            return text

    def _normalize_word_vowel_ending_dravidian(self, word):
        """
        for Dravidian
        - consonant ending: add 'a' ki maatra
        - halant ending: no change
        - 'a' ki maatra: no change
        """
        if len(word) > 0 and langinfo.is_consonant(word[-1], self.lang):
            return word + langinfo.offset_to_char(0x3E, self.lang)
        else:
            return word

    def _normalize_word_vowel_ending_ie(self, word):
        """
        for IE
        - consonant ending: add halant
        - halant ending: no change
        - 'a' ki maatra: no change
        """
        if len(word) > 0 and langinfo.is_consonant(word[-1], self.lang):
            return word + langinfo.offset_to_char(langinfo.HALANTA_OFFSET, self.lang)
        else:
            return word

    def _normalize_vowel_ending(self, text):
        return " ".join([self.fn_vowel_ending(w) for w in text.split(" ")])

    def normalize(self, text):
        """
        Method to be implemented for normalization for each script
        """
        text = text.replace(NormalizerI.BYTE_ORDER_MARK, "")
        text = text.replace(NormalizerI.BYTE_ORDER_MARK_2, "")
        text = text.replace(NormalizerI.WORD_JOINER, "")
        text = text.replace(NormalizerI.SOFT_HYPHEN, "")

        text = text.replace(NormalizerI.ZERO_WIDTH_SPACE, " ")  # ??
        text = text.replace(NormalizerI.NO_BREAK_SPACE, " ")

        text = text.replace(NormalizerI.ZERO_WIDTH_NON_JOINER, "")
        text = text.replace(NormalizerI.ZERO_WIDTH_JOINER, "")

        text = self._normalize_punctuations(text)

        if self.do_normalize_chandras:
            text = self._normalize_chandras(text)
        text = self._normalize_nasals(text)
        if self.do_normalize_vowel_ending:
            text = self._normalize_vowel_ending(text)

        return text

    def get_char_stats(self, text):
        print(len(re.findall(NormalizerI.BYTE_ORDER_MARK, text)))
        print(len(re.findall(NormalizerI.BYTE_ORDER_MARK_2, text)))
        print(len(re.findall(NormalizerI.WORD_JOINER, text)))
        print(len(re.findall(NormalizerI.SOFT_HYPHEN, text)))

        print(len(re.findall(NormalizerI.ZERO_WIDTH_SPACE, text)))
        print(len(re.findall(NormalizerI.NO_BREAK_SPACE, text)))

        print(len(re.findall(NormalizerI.ZERO_WIDTH_NON_JOINER, text)))
        print(len(re.findall(NormalizerI.ZERO_WIDTH_JOINER, text)))

        # for mobj in re.finditer(NormalizerI.ZERO_WIDTH_NON_JOINER,text):
        #    print text[mobj.start()-10:mobj.end()+10].replace('\n', ' ').replace(NormalizerI.ZERO_WIDTH_NON_JOINER,'').encode('utf-8')
        # print hex(ord(text[mobj.end():mobj.end()+1]))

    def correct_visarga(self, text, visarga_char, char_range):
        text = re.sub(r"([\u0900-\u097f]):", "\\1\u0903", text)

In [ ]:
# | export
class DevanagariNormalizer(BaseNormalizer):
    """
    Normalizer for the Devanagari script. In addition to basic normalization by the super class,
    * Replaces the composite characters containing nuktas by their decomposed form
    * replace pipe character '|' by poorna virama character
    * replace colon ':' by visarga if the colon follows a charcter in this script

    """

    NUKTA = "\u093c"

    def __init__(
        self,
        lang="hi",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
    ):
        super(DevanagariNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )

    def __call__(self, text):

        # common normalization for Indic scripts
        text = super(DevanagariNormalizer, self).normalize(text)

        # chandra a replacement for Marathi
        text = text.replace("\u0972", "\u090f")

        # decomposing Nukta based composite characters
        text = text.replace("\u0929", "\u0928" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0931", "\u0930" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0934", "\u0933" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0958", "\u0915" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0959", "\u0916" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095a", "\u0917" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095b", "\u091c" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095c", "\u0921" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095d", "\u0922" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095e", "\u092b" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095f", "\u092f" + DevanagariNormalizer.NUKTA)

        if self.remove_nuktas:
            text = text.replace(DevanagariNormalizer.NUKTA, "")

        # replace pipe character for poorna virama
        text = text.replace("\u007c", "\u0964")

        # correct visarga
        text = re.sub(r"([\u0900-\u097f]):", "\\1\u0903", text)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            # Extract parts containing digits
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="hi"))

        return text

    def get_char_stats(self, text):
        super(DevanagariNormalizer, self).get_char_stats(text)

        print((len(re.findall("\u0929", text))))
        print((len(re.findall("\u0931", text))))
        print((len(re.findall("\u0934", text))))
        print((len(re.findall("\u0958", text))))
        print((len(re.findall("\u0959", text))))
        print((len(re.findall("\u095a", text))))
        print((len(re.findall("\u095b", text))))
        print((len(re.findall("\u095c", text))))
        print((len(re.findall("\u095d", text))))
        print((len(re.findall("\u095e", text))))
        print((len(re.findall("\u095f", text))))

In [ ]:
norm = DevanagariNormalizer()
TEST_RESULT = "चीर धाराएं समुद्र तट पर लगकर लौटने वाली लहरों का प्रवाह होता है अक्सर एक चट्टान या इसी तरह के पदार्थों पर"
hi_text = "चीर धाराएं समुद्र तट पर लगकर लौटने वाली लहरों का प्रवाह होता है अक्सर एक चट्टान या इसी तरह के पदार्थों पर"
norm(hi_text)

'चीर धाराएं समुद्र तट पर लगकर लौटने वाली लहरों का प्रवाह होता है अक्सर एक चट्टान या इसी तरह के पदार्थों पर'

In [ ]:
assert norm(hi_text) == TEST_RESULT

In [ ]:
sample_text = "भारत का सकल घरेलू उत्पाद 1.5 ट्रिलियन अमेरिकी डॉलर है।"
norm(sample_text)

'भारत का सकल घरेलू उत्पाद एक.पाँच ट्रिलियन अमेरिकी डॉलर है।'

In [ ]:
#|export
class HindiNormalizer(BaseNormalizer):
    """Fork of Devanagiri normalizer. With additional changes for Hindi and tts_mode."""

    NUKTA = "\u093c"

    def __init__(
        self,
        lang="hi",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        tts_mode=False,
    ):
        super(HindiNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.tts_mode = tts_mode

    def __call__(self, text):

        # common normalization for Indic scripts
        text = super(HindiNormalizer, self).normalize(text)

        # chandra a replacement for Marathi
        text = text.replace("\u0972", "\u090f")

        # decomposing Nukta based composite characters
        text = text.replace("\u0929", "\u0928" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0931", "\u0930" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0934", "\u0933" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0958", "\u0915" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u0959", "\u0916" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095a", "\u0917" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095b", "\u091c" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095c", "\u0921" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095d", "\u0922" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095e", "\u092b" + DevanagariNormalizer.NUKTA)
        text = text.replace("\u095f", "\u092f" + DevanagariNormalizer.NUKTA)

        if self.remove_nuktas:
            text = text.replace(DevanagariNormalizer.NUKTA, "")

        # replace pipe character for poorna virama
        text = text.replace("\u007c", "\u0964")
        # correct visarga
        text = re.sub(r"([\u0900-\u097f]):", "\\1\u0903", text)

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"रुपये \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"रुपये \1", text)
            text = re.sub(r"₹\s*(\d+)", r"रुपये \1", text)
            text = re.sub(r"USD\s+(\d+)", r"डॉलर \1", text)
            text = re.sub(r"\$\s*(\d+)", r"डॉलर \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"कोरियाई वॉन \1", text)
            text = re.sub(r"₩\s*(\d+)", r"कोरियाई वॉन \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} पॉइंट {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "एच टी टी पी एस कोलन स्लैश स्लैश ")
                url = url.replace("http://", "एच टी टी पी कोलन स्लैश स्लैश ")

                # Break domain name only, not TLD
                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " डॉट " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" स्लैश " + rest if rest else "")

                url = url.replace(".", " डॉट ")
                url = url.replace("/", " स्लैश ")
                url = url.replace(":", " कोलन ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)
            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} एट {m.group(2)} डॉट {m.group(3)}",
                text,
            )

            # Replace standalone special characters
            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }
            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="hi"))
        return text

In [ ]:
norm = HindiNormalizer(tts_mode=True)
TEST_RESULT = "चीर धाराएं समुद्र तट पर लगकर लौटने वाली लहरों का प्रवाह होता है अक्सर एक चट्टान या इसी तरह के पदार्थों पर"
hi_text = "चीर धाराएं समुद्र तट पर लगकर लौटने वाली लहरों का प्रवाह होता है अक्सर एक चट्टान या इसी तरह के पदार्थों पर $134.5"
norm(hi_text)

'चीर धाराएं समुद्र तट पर लगकर लौटने वाली लहरों का प्रवाह होता है अक्सर एक चट्टान या इसी तरह के पदार्थों पर डॉलर एक सौ चौंतीस पॉइंट पाँच'

In [ ]:
normalizer = HindiNormalizer(tts_mode=True)
output = normalizer("He spent Rs. 500 and $20 on groceries.")
print(output)

He spent रुपये पाँच सौ and डॉलर बीस on groceries.


In [ ]:
normalizer = HindiNormalizer(tts_mode=True)
output = normalizer(
    "Visit https://example.com or mail us at help@support.in & get 20% off!"
)
print(output)

Visit एच टी टी पी एस कोलन स्लैश स्लैश e x a m p l e डॉट com or mail us at help एट support डॉट in  or  get बीस percent  off!


In [ ]:
# | export
class PunjabiNormalizer(BaseNormalizer):
    """
    Normalizer for the Gurmukhi script. In addition to basic normalization by the super class,
    * Replaces the composite characters containing nuktas by their decomposed form
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * replace pipe character '|' by poorna virama character
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    NUKTA = "\u0a3c"

    VOWEL_NORM_MAPS = {
        ## http://www.unicode.org/versions/Unicode12.1.0/ch12.pdf
        ## Table 12-16
        "\u0a05\u0a3e": "\u0a06",
        "\u0a72\u0a3f": "\u0a07",
        "\u0a72\u0a40": "\u0a08",
        "\u0a73\u0a41": "\u0a09",
        "\u0a73\u0a42": "\u0a0a",
        "\u0a72\u0a47": "\u0a0f",
        "\u0a05\u0a48": "\u0a10",
        "\u0a73\u0a4b": "\u0a13",
        "\u0a05\u0a4c": "\u0a14",
    }

    def __init__(
        self,
        lang="pa",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        do_canonicalize_addak=False,
        do_canonicalize_tippi=False,
        do_replace_vowel_bases=False,
        tts_mode=False,
    ):
        super(PunjabiNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.do_canonicalize_addak = do_canonicalize_addak
        self.do_canonicalize_tippi = do_canonicalize_tippi
        self.do_replace_vowel_bases = do_replace_vowel_bases
        self.tts_mode = tts_mode

    def _normalize_vowels(self, text):
        """ """

        ## standard vowel replacements as per suggestions in
        ## http://www.unicode.org/versions/Unicode12.1.0/ch12.pdf
        ## Table 12-16

        for k, v in PunjabiNormalizer.VOWEL_NORM_MAPS.items():
            text = text.replace(k, v)

        ## the above mappings should account for majority of the variantions,
        ## Rest are handled via this generic rule which looks at the diacritic
        ## following the 2 special characters
        ## TBD: don't see evidence for this in Wikipedia corpus

        ## If these special characters occur without any diacritic, replace them with closet
        ## equivalent vowels
        if self.do_replace_vowel_bases:
            text = text.replace("\u0a72", "\u0a07")
            text = text.replace("\u0a73", "\u0a09")

        return text

    def __call__(self, text):

        # Addak
        if self.do_canonicalize_addak:
            ## replace addak+consonant with consonat+halant+consonant
            text = re.sub(r"\u0a71(.)", "\\1\u0a4d\\1", text)

        # Tippi
        if self.do_canonicalize_tippi:
            text = text.replace("\u0a70", "\u0a02")

        # Vowels: Gurumuki has multiple ways of representing independent vowels due
        # to the characters 'iri' and 'ura'.
        text = self._normalize_vowels(text)

        # common normalization for Indic scripts
        text = super(PunjabiNormalizer, self).normalize(text)

        # decomposing Nukta based composite characters
        text = text.replace("\u0a33", "\u0a32" + PunjabiNormalizer.NUKTA)
        text = text.replace("\u0a36", "\u0a38" + PunjabiNormalizer.NUKTA)
        text = text.replace("\u0a59", "\u0a16" + PunjabiNormalizer.NUKTA)
        text = text.replace("\u0a5a", "\u0a17" + PunjabiNormalizer.NUKTA)
        text = text.replace("\u0a5b", "\u0a1c" + PunjabiNormalizer.NUKTA)
        text = text.replace("\u0a5e", "\u0a2b" + PunjabiNormalizer.NUKTA)

        if self.remove_nuktas:
            text = text.replace(PunjabiNormalizer.NUKTA, "")

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u0a64", "\u0964")
        text = text.replace("\u0a65", "\u0965")

        ## replace pipe character for poorna virama
        text = text.replace("\u007c", "\u0964")

        # correct visarge
        text = re.sub(r"([\u0a00-\u0a7f]):", "\\1\u0a03", text)

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"ਰੁਪਏ \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"ਰੁਪਏ \1", text)
            text = re.sub(r"₹\s*(\d+)", r"ਰੁਪਏ \1", text)
            text = re.sub(r"USD\s+(\d+)", r"ਡਾਲਰ \1", text)
            text = re.sub(r"\$\s*(\d+)", r"ਡਾਲਰ \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"ਕੋਰੀਆਈ ਵੌਨ \1", text)
            text = re.sub(r"₩\s*(\d+)", r"ਕੋਰੀਆਈ ਵੌਨ \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} ਪੌਇੰਟ {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "ਐਚ ਟੀ ਟੀ ਪੀ ਐੱਸ ਕੋਲਨ ਸਲੈਸ਼ ਸਲੈਸ਼ ")
                url = url.replace("http://", "ਐਚ ਟੀ ਟੀ ਪੀ ਕੋਲਨ ਸਲੈਸ਼ ਸਲੈਸ਼ ")

                # Break domain name only, not TLD
                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " ਡਾਟ " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" ਸਲੈਸ਼ " + rest if rest else "")

                url = url.replace(".", " ਡਾਟ ")
                url = url.replace("/", " ਸਲੈਸ਼ ")
                url = url.replace(":", " ਕੋਲਨ ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} ਐਟ {m.group(2)} ਡਾਟ {m.group(3)}",
                text,
            )

            # Replace standalone special characters
            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }
            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="pa"))

        return text

In [ ]:
normalizer = PunjabiNormalizer(tts_mode=True)
punjabi_text = "ਪੰਜੀਵੀ ਜੀਡੀਪੀ 1.5 ਟ੍ਰੀਲੀਅਨ ਅਮਰੀਕਾ ਡੋਲਰ ਛੇ।"
normalizer(punjabi_text)

'ਪੰਜੀਵੀ ਜੀਡੀਪੀ ਇੱਕ ਪੌਇੰਟ ਪੰਜ ਟ੍ਰੀਲੀਅਨ ਅਮਰੀਕਾ ਡੋਲਰ ਛੇ।'

In [ ]:
punjabi_text = (
    "ਸਤਿ ਸ੍ਰੀ ਅਕਾਲ ਦੁਨੀਆਂ। 4 ਮਈ, 2025 ਨੂੰ ਰਿਲੀਜ਼ ਹੋਏ ਪੰਜਾਬੀ ਨੋਰਮਲਾਈਜ਼ਰ ਵਿੱਚ ਤੁਹਾਡਾ ਸਵਾਗਤ ਹੈ।"
)
normalizer(punjabi_text)

'ਸਤਿ ਸ੍ਰੀ ਅਕਾਲ ਦੁਨੀਆਂ। ਚਾਰ ਮਈ, ਦੋ ਹਜ਼ਾਰ ਪੱਚੀ ਨੂੰ ਰਿਲੀਜ਼ ਹੋਏ ਪੰਜਾਬੀ ਨੋਰਮਲਾਈਜ਼ਰ ਵਿੱਚ ਤੁਹਾਡਾ ਸਵਾਗਤ ਹੈ।'

In [ ]:
# | export
class TeluguNormalizer(BaseNormalizer):
    """
    Normalizer for the Teluguscript. In addition to basic normalization by the super class,
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * canonicalize two-part dependent vowel signs
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    def __init__(
        self,
        lang="te",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        tts_mode=False,
    ):
        super(TeluguNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.tts_mode = tts_mode

    def __call__(self, text: str):

        # common normalization for Indic scripts
        text = super(TeluguNormalizer, self).normalize(text)

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u0c64", "\u0964")
        text = text.replace("\u0c65", "\u0965")

        # dependent vowels
        text = text.replace("\u0c46\u0c56", "\u0c48")

        # correct visarge
        text = re.sub(r"([\u0c00-\u0c7f]):", "\\1\u0c03", text)

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"రూపాయలు \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"రూపాయలు \1", text)
            text = re.sub(r"₹\s*(\d+)", r"రూపాయలు \1", text)
            text = re.sub(r"USD\s+(\d+)", r"డాలర్లు \1", text)
            text = re.sub(r"\$\s*(\d+)", r"డాలర్లు \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"కొరియన్ వాన్ \1", text)
            text = re.sub(r"₩\s*(\d+)", r"కొరియన్ వాన్ \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} పాయింట్ {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "హెచ్ టి టి పి ఎస్ కొలన్ స్లాష్ స్లాష్ ")
                url = url.replace("http://", "హెచ్ టి టి పి కొలన్ స్లాష్ స్లాష్ ")

                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " డాట్ " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" స్లాష్ " + rest if rest else "")

                url = url.replace(".", " డాట్ ")
                url = url.replace("/", " స్లాష్ ")
                url = url.replace(":", " కొలన్ ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} అట్ {m.group(2)} డాట్ {m.group(3)}",
                text,
            )

            # Replace standalone special characters
            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }
            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="te"))

        return text

    def get_char_stats(self, text):
        pass

In [ ]:
norm = TeluguNormalizer()
te_text = "భారతదేశ జీడీపీ 1.5 ట్రిలియన్ అమెరికా డాలర్లు."
norm(te_text)

'భారతదేశ జీడీపీ ఒకటి.ఐదు ట్రిలియన్ అమెరికా డాలర్లు.'

In [ ]:
# | export
class GujaratiNormalizer(BaseNormalizer):
    """
    Normalizer for the Gujarati script. In addition to basic normalization by the super class,
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    NUKTA = "\u0abc"

    def __init__(
        self,
        lang="gu",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        tts_mode=False,
    ):
        super(GujaratiNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.tts_mode = tts_mode

    def __call__(self, text: str):

        # common normalization for Indic scripts
        text = super(GujaratiNormalizer, self).normalize(text)

        # decomposing Nukta based composite characters
        if self.remove_nuktas:
            text = text.replace(GujaratiNormalizer.NUKTA, "")

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u0ae4", "\u0964")
        text = text.replace("\u0ae5", "\u0965")

        # correct visarge
        text = re.sub(r"([\u0a80-\u0aff]):", "\\1\u0a83", text)

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"રૂપિયા \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"રૂપિયા \1", text)
            text = re.sub(r"₹\s*(\d+)", r"રૂપિયા \1", text)
            text = re.sub(r"USD\s+(\d+)", r"ડોલર \1", text)
            text = re.sub(r"\$\s*(\d+)", r"ડોલર \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"કોરિયન વોન \1", text)
            text = re.sub(r"₩\s*(\d+)", r"કોરિયન વોન \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} પોઈન્ટ {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "એચ ટી ટી પીએસ કોલન સ્લેશ સ્લેશ ")
                url = url.replace("http://", "એચ ટી ટી પી કોલન સ્લેશ સ્લેશ ")

                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " ડોટ " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" સ્લેશ " + rest if rest else "")

                url = url.replace(".", " ડોટ ")
                url = url.replace("/", " સ્લેશ ")
                url = url.replace(":", " કોલન ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} એટ {m.group(2)} ડોટ {m.group(3)}",
                text,
            )

            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }
            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="gu"))

        return text

In [ ]:
norm = GujaratiNormalizer(tts_mode=True)
gujarati_text = "ભારતનો જીડીપી 1.5 ટ્રિલિયન અમેરિકન ડોલર છે."
norm(gujarati_text)

'ભારતનો જીડીપી એક પોઈન્ટ પાંચ ટ્રિલિયન અમેરિકન ડોલર છે.'

In [ ]:
# | export
class OdiaNormalizer(BaseNormalizer):
    """
    Normalizer for the Oriya script. In addition to basic normalization by the super class,
    * Replaces the composite characters containing nuktas by their decomposed form
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * Canonicalize two part dependent vowels
    * Replace 'va' with 'ba'
    * replace pipe character '|' by poorna virama character
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    NUKTA = "\u0b3c"

    VOWEL_NORM_MAPS = {
        ## See Table 12-22 in http://www.unicode.org/versions/Unicode12.1.0/ch12.pdf
        "\u0b05\u0b3e": "\u0b06",
        "\u0b0f\u0b57": "\u0b10",
        "\u0b13\u0b57": "\u0b14",
    }

    def __init__(
        self,
        lang="or",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        do_remap_wa=False,
        tts_mode=False,
    ):
        super(OdiaNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.do_remap_wa = do_remap_wa
        self.tts_mode = tts_mode

    def __call__(self, text: str):

        # common normalization for Indic scripts
        text = super(OdiaNormalizer, self).normalize(text)

        ## standard vowel replacements as per suggestions in Unicode documents
        for k, v in OdiaNormalizer.VOWEL_NORM_MAPS.items():
            text = text.replace(k, v)

        # decomposing Nukta based composite characters
        text = text.replace("\u0b5c", "\u0b21" + OdiaNormalizer.NUKTA)
        text = text.replace("\u0b5d", "\u0b22" + OdiaNormalizer.NUKTA)

        if self.remove_nuktas:
            text = text.replace(OdiaNormalizer.NUKTA, "")

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u0b64", "\u0964")
        text = text.replace("\u0b65", "\u0965")

        # replace pipe character for poorna virama
        text = text.replace("\u0b7c", "\u0964")

        # replace wa with ba
        if self.do_remap_wa:
            text = text.replace("\u0b71", "\u0b2c")

        # replace va with ba
        # NOTE: documentation (chapter on Indic scripts) and codepoint chart seem contradictory
        # (this applied to wa to ba rule also above)
        text = text.replace("\u0b35", "\u0b2c")

        # AI dependent vowel sign
        text = text.replace("\u0b47\u0b56", "\u0b58")

        # two part dependent vowels
        text = text.replace("\u0b47\u0b3e", "\u0b4b")
        text = text.replace("\u0b47\u0b57", "\u0b4c")

        # additional consonant - not clear how to handle this
        # ignore

        # correct visarge
        text = re.sub(r"([\u0b00-\u0b7f]):", "\\1\u0b03", text)

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"ଟଙ୍କା \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"ଟଙ୍କା \1", text)
            text = re.sub(r"₹\s*(\d+)", r"ଟଙ୍କା \1", text)
            text = re.sub(r"USD\s+(\d+)", r"ଡଲାର \1", text)
            text = re.sub(r"\$\s*(\d+)", r"ଡଲାର \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"କୋରିଆନ୍ ୱନ୍ \1", text)
            text = re.sub(r"₩\s*(\d+)", r"କୋରିଆନ୍ ୱନ୍ \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} ପଏଣ୍ଟ {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "ଏଚ୍ ଟି ଟି ପି ଏସ୍ କୋଲନ୍ ସ୍ଲାଶ୍ ସ୍ଲାଶ୍ ")
                url = url.replace("http://", "ଏଚ୍ ଟି ଟି ପି କୋଲନ୍ ସ୍ଲାଶ୍ ସ୍ଲାଶ୍ ")

                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " ଡଟ୍ " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" ସ୍ଲାଶ୍ " + rest if rest else "")

                url = url.replace(".", " ଡଟ୍ ")
                url = url.replace("/", " ସ୍ଲାଶ୍ ")
                url = url.replace(":", " କୋଲନ୍ ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} ଏଟ୍ {m.group(2)} ଡଟ୍ {m.group(3)}",
                text,
            )

            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }

            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="or"))

        return text

In [ ]:
norm = OdiaNormalizer(tts_mode=True)
odia_text = "ଭାରତର ଜିଡିପି 1.5 ଟ୍ରିଲିଅନ୍ ଆମେରିକୀୟ ଡଲାର ଅଟେ।"
norm(odia_text)

'ଭାରତର ଜିଡିପି ଏକ ପଏଣ୍ଟ ପାଞ୍ଚ ଟ୍ରିଲିଅନ୍ ଆମେରିକୀୟ ଡଲାର ଅଟେ।'

In [ ]:
# | export
class BengaliNormalizer(BaseNormalizer):
    """
    Normalizer for the Bengali script. In addition to basic normalization by the super class,
    * Replaces the composite characters containing nuktas by their decomposed form
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * Canonicalize two part dependent vowels
    * replace pipe character '|' by poorna virama character
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    NUKTA = "\u09bc"

    def __init__(
        self,
        lang="bn",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        do_remap_assamese_chars=False,
        tts_mode=False,
    ):
        super(BengaliNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.do_remap_assamese_chars = do_remap_assamese_chars
        self.tts_mode = True

    def __call__(self, text: str):

        # common normalization for Indic scripts
        text = super(BengaliNormalizer, self).normalize(text)

        # decomposing Nukta based composite characters
        text = text.replace("\u09dc", "\u09a1" + BengaliNormalizer.NUKTA)
        text = text.replace("\u09dd", "\u09a2" + BengaliNormalizer.NUKTA)
        text = text.replace("\u09df", "\u09af" + BengaliNormalizer.NUKTA)

        if self.remove_nuktas:
            text = text.replace(BengaliNormalizer.NUKTA, "")

        if self.do_remap_assamese_chars and self.lang == "as":
            text = text.replace("\u09f0", "\u09b0")  #  'ra' character
            text = text.replace("\u09f1", "\u09ac")  #  'va' character

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u09e4", "\u0964")
        text = text.replace("\u09e5", "\u0965")

        # replace pipe character for poorna virama
        text = text.replace("\u007c", "\u0964")
        # replace bengali currency numerator four for poorna virama  (it looks similar and is used as a substitute)
        text = text.replace("\u09f7", "\u0964")

        # two part dependent vowels
        text = text.replace("\u09c7\u09be", "\u09cb")
        text = text.replace("\u09c7\u09d7", "\u09cc")

        # correct visarge
        text = re.sub(r"([\u0980-\u09ff]):", "\\1\u0983", text)

        if self.tts_mode:
            text = re.sub(r"INR\s+(\d+)", r"রুপি \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"রুপি \1", text)
            text = re.sub(r"₹\s*(\d+)", r"রুপি \1", text)
            text = re.sub(r"USD\s+(\d+)", r"ডলার \1", text)
            text = re.sub(r"\$\s*(\d+)", r"ডলার \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"কোরিয়ান ওন \1", text)
            text = re.sub(r"₩\s*(\d+)", r"কোরিয়ান ওন \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} পয়েন্ট {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "এইচ টি টি পি এস কোলন স্ল্যাশ স্ল্যাশ ")
                url = url.replace("http://", "এইচ টি টি পি কোলন স্ল্যাশ স্ল্যাশ ")

                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " ডট " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" স্ল্যাশ " + rest if rest else "")

                url = url.replace(".", " ডট ")
                url = url.replace("/", " স্ল্যাশ ")
                url = url.replace(":", " কোলন ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} অ্যাট {m.group(2)} ডট {m.group(3)}",
                text,
            )

            # Replace standalone special characters
            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }

            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="bn"))

        return text

In [ ]:
norm = BengaliNormalizer(tts_mode=True)

TEST_RESULT = "ভারতের জিডিপি ১.৫ ট্রিলিয়ন মার্কিন ডলার।"  # Claude generated output
bn_text = "ভারতের জিডিপি 1.5 ট্রিলিয়ন মার্কিন ডলার।"
norm(bn_text)

'ভারতের জিডিপি এক পয়েন্ট পাঁচ ট্রিলিয়ন মার্কিন ডলার।'

In [ ]:
# | export
class TamilNormalizer(BaseNormalizer):
    """
    Normalizer for the Tamil script. In addition to basic normalization by the super class,
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * canonicalize two-part dependent vowel signs
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    def __init__(
        self,
        lang="ta",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        tts_mode=False,
    ):
        super(TamilNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.tts_mode = tts_mode

    def __call__(self, text: str):

        # common normalization for Indic scripts
        text = super(TamilNormalizer, self).normalize(text)

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u0be4", "\u0964")
        text = text.replace("\u0be5", "\u0965")

        # two part dependent vowels
        text = text.replace("\u0b92\u0bd7", "\u0b94")
        text = text.replace("\u0bc6\u0bbe", "\u0bca")
        text = text.replace("\u0bc7\u0bbe", "\u0bcb")
        text = text.replace("\u0bc6\u0bd7", "\u0bcc")

        # correct visarge
        text = re.sub(r"([\u0b80-\u0bff]):", "\\1\u0b83", text)

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"ரூபாய் \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"ரூபாய் \1", text)
            text = re.sub(r"₹\s*(\d+)", r"ரூபாய் \1", text)
            text = re.sub(r"USD\s+(\d+)", r"டாலர் \1", text)
            text = re.sub(r"\$\s*(\d+)", r"டாலர் \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"கொரிய வான் \1", text)
            text = re.sub(r"₩\s*(\d+)", r"கொரிய வான் \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} பாயிண்ட் {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "எச் டி டி பி எஸ் கோலன் ஸ்லாஷ் ஸ்லாஷ் ")
                url = url.replace("http://", "எச் டி டி பி கோலன் ஸ்லாஷ் ஸ்லாஷ் ")

                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " டாட் " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" ஸ்லாஷ் " + rest if rest else "")

                url = url.replace(".", " டாட் ")
                url = url.replace("/", " ஸ்லாஷ் ")
                url = url.replace(":", " கோலன் ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} அட் {m.group(2)} டாட் {m.group(3)}",
                text,
            )

            # Replace standalone special characters
            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }

            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="ta"))

        return text

In [ ]:
norm = TamilNormalizer(tts_mode=True)
ta_text = "இந்தியாவின் மொத்த உள்நாட்டு உற்பத்தி 1.5 டிரில்லியன் அமெரிக்க டாலர்."
norm(ta_text)

'இந்தியாவின் மொத்த உள்நாட்டு உற்பத்தி ஒன்று பாயிண்ட் ஐந்து டிரில்லியன் அமெரிக்க டாலர்.'

In [ ]:
# | export
class KannadaNormalizer(BaseNormalizer):
    """
    Normalizer for the Kannada script. In addition to basic normalization by the super class,
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * canonicalize two-part dependent vowel signs
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    def __init__(
        self,
        lang="kn",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        tts_mode=False,
    ):
        super(KannadaNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.tts_mode = tts_mode

    def __call__(self, text: str):

        # common normalization for Indic scripts
        text = super(KannadaNormalizer, self).normalize(text)

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u0ce4", "\u0964")
        text = text.replace("\u0ce5", "\u0965")

        # dependent vowels
        text = text.replace("\u0cbf\u0cd5", "\u0cc0")
        text = text.replace("\u0cc6\u0cd5", "\u0cc7")
        text = text.replace("\u0cc6\u0cd6", "\u0cc8")
        text = text.replace("\u0cc6\u0cc2", "\u0cca")
        text = text.replace("\u0cca\u0cd5", "\u0ccb")

        # correct visarge
        text = re.sub(r"([\u0c80-\u0cff]):", "\\1\u0c83", text)

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"ರೂಪಾಯಿ \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"ರೂಪಾಯಿ \1", text)
            text = re.sub(r"₹\s*(\d+)", r"ರೂಪಾಯಿ \1", text)
            text = re.sub(r"USD\s+(\d+)", r"ಡಾಲರ್ \1", text)
            text = re.sub(r"\$\s*(\d+)", r"ಡಾಲರ್ \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"ಕೊರಿಯನ್ ವಾನ್ \1", text)
            text = re.sub(r"₩\s*(\d+)", r"ಕೊರಿಯನ್ ವಾನ್ \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} ಪಾಯಿಂಟ್ {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "ಎಚ್ ಟಿ ಟಿ ಪಿ ಎಸ್ ಕೋಲನ್ ಸ್ಲ್ಯಾಶ್ ಸ್ಲ್ಯಾಶ್ ")
                url = url.replace("http://", "ಎಚ್ ಟಿ ಟಿ ಪಿ ಕೋಲನ್ ಸ್ಲ್ಯಾಶ್ ಸ್ಲ್ಯಾಶ್ ")

                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " ಡಾಟ್ " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" ಸ್ಲ್ಯಾಶ್ " + rest if rest else "")

                url = url.replace(".", " ಡಾಟ್ ")
                url = url.replace("/", " ಸ್ಲ್ಯಾಶ್ ")
                url = url.replace(":", " ಕೋಲನ್ ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} ಅಟ್ {m.group(2)} ಡಾಟ್ {m.group(3)}",
                text,
            )

            # Replace standalone special characters
            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }
            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="kn"))

        return text

In [ ]:
norm = KannadaNormalizer(tts_mode=True)
kannada_text = "ಭಾರತದ ಜಿಡಿಪಿ 1.5 ಟ್ರಿಲಿಯನ್ ಅಮೇರಿಕನ್ ಡಾಲರ್ ಆಗಿದೆ."
norm(kannada_text)

'ಭಾರತದ ಜಿಡಿಪಿ ಒಂದು ಪಾಯಿಂಟ್ ಐದು ಟ್ರಿಲಿಯನ್ ಅಮೇರಿಕನ್ ಡಾಲರ್ ಆಗಿದೆ.'

In [ ]:
# | export
class MalayalamNormalizer(BaseNormalizer):
    """
    Normalizer for the Malayalam script. In addition to basic normalization by the super class,
    * Replace the reserved character for poorna virama (if used) with the recommended generic Indic scripts poorna virama
    * canonicalize two-part dependent vowel signs
    * Change from old encoding of chillus (till Unicode 5.0) to new encoding
    * replace colon ':' by visarga if the colon follows a charcter in this script
    """

    CHILLU_CHAR_MAP = {
        "\u0d7a": "\u0d23",
        "\u0d7b": "\u0d28",
        "\u0d7c": "\u0d30",
        "\u0d7d": "\u0d32",
        "\u0d7e": "\u0d33",
        "\u0d7f": "\u0d15",
    }

    def _canonicalize_chillus(self, text):
        for chillu, char in MalayalamNormalizer.CHILLU_CHAR_MAP.items():
            text = text.replace(chillu, "{}\u0d4d".format(char))
        return text

    def _correct_geminated_T(self, text):
        return text.replace("\u0d31\u0d4d\u0d31", "\u0d1f\u0d4d\u0d1f")

    def __init__(
        self,
        lang="ml",
        remove_nuktas=False,
        nasals_mode="do_nothing",
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False,
        do_canonicalize_chillus=False,
        do_correct_geminated_T=False,
        tts_mode=False,
    ):
        super(MalayalamNormalizer, self).__init__(
            lang,
            remove_nuktas,
            nasals_mode,
            do_normalize_chandras,
            do_normalize_vowel_ending,
        )
        self.do_canonicalize_chillus = do_canonicalize_chillus
        self.do_correct_geminated_T = do_correct_geminated_T
        self.tts_mode = tts_mode

    def __call__(self, text: str):
        # Change from old encoding of chillus (till Unicode 5.0) to new encoding
        text = text.replace("\u0d23\u0d4d\u200d", "\u0d7a")
        text = text.replace("\u0d28\u0d4d\u200d", "\u0d7b")
        text = text.replace("\u0d30\u0d4d\u200d", "\u0d7c")
        text = text.replace("\u0d32\u0d4d\u200d", "\u0d7d")
        text = text.replace("\u0d33\u0d4d\u200d", "\u0d7e")
        text = text.replace("\u0d15\u0d4d\u200d", "\u0d7f")

        # Normalize chillus
        if self.do_canonicalize_chillus:
            text = self._canonicalize_chillus(text)

        # common normalization for Indic scripts
        text = super(MalayalamNormalizer, self).normalize(text)

        # replace the poorna virama codes specific to script
        # with generic Indic script codes
        text = text.replace("\u0d64", "\u0964")
        text = text.replace("\u0d65", "\u0965")

        # dependent vowels
        text = text.replace("\u0d46\u0d3e", "\u0d4a")
        text = text.replace("\u0d47\u0d3e", "\u0d4b")

        # au forms
        text = text.replace("\u0d46\u0d57", "\u0d4c")
        text = text.replace("\u0d57", "\u0d4c")

        # correct geminated T
        if self.do_correct_geminated_T:
            text = self._correct_geminated_T(text)

        # correct visarga
        text = re.sub(r"([\u0d00-\u0d7f]):", "\\1\u0d03", text)
        # remove samvruthokaram
        text = text.replace("\u0d41\u0d4d", "\u0d4d")

        if self.tts_mode:
            # Handle currencies
            text = re.sub(r"INR\s+(\d+)", r"രൂപ \1", text)
            text = re.sub(r"Rs\.\s+(\d+)", r"രൂപ \1", text)
            text = re.sub(r"₹\s*(\d+)", r"രൂപ \1", text)
            text = re.sub(r"USD\s+(\d+)", r"ഡോളർ \1", text)
            text = re.sub(r"\$\s*(\d+)", r"ഡോളർ \1", text)
            text = re.sub(r"KRW\s+(\d+)", r"കൊറിയൻ വോൺ \1", text)
            text = re.sub(r"₩\s*(\d+)", r"കൊറിയൻ വോൺ \1", text)

            # Handle decimal numbers
            def replace_decimal(match):
                whole, frac = match.group(1), match.group(2)
                return f"{whole} പോയിന്റ് {frac}"

            text = re.sub(r"(\d+)\.(\d+)", replace_decimal, text)

            # Normalize URLs before replacing symbols
            def normalize_url(match):
                url = match.group(0)
                url = url.replace("https://", "എച്ച് ടി ടി പി എസ് കോളൺ സ്ലാഷ് സ്ലാഷ് ")
                url = url.replace("http://", "എച്ച് ടി ടി പി കോളൺ സ്ലാഷ് സ്ലാഷ് ")

                domain_parts = url.split("/")
                domain = domain_parts[0]
                domain = re.sub(
                    r"([a-zA-Z]+)\.([a-z]{2,3})(?=\b|/)",
                    lambda m: " ".join(m.group(1)) + " ഡോട്ട് " + m.group(2),
                    domain,
                )

                rest = "/".join(domain_parts[1:])
                url = domain + (" സ്ലാഷ് " + rest if rest else "")

                url = url.replace(".", " ഡോട്ട് ")
                url = url.replace("/", " സ്ലാഷ് ")
                url = url.replace(":", " കോളൺ ")
                return url.strip()

            text = re.sub(r"https?://[^\s]+", normalize_url, text)

            # Email normalization
            text = re.sub(
                r"\b([\w\.-]+)@([\w\.-]+)\.(\w+)\b",
                lambda m: f"{m.group(1)} അറ്റ് {m.group(2)} ഡോട്ട് {m.group(3)}",
                text,
            )

            # Replace standalone special characters
            symbol_map = {
                "&": " or ",
                "@": " at ",
                "%": " percent ",
                "+": " plus ",
                "-": " minus ",
                "=": " equals ",
                "*": " star ",
                "#": " hash ",
            }
            for sym, word in symbol_map.items():
                text = text.replace(sym, word)

        has_digits = any(char.isdigit() for char in text)
        if has_digits:
            digit_parts = re.findall(r"\d+", text)
            for part in digit_parts:
                text = text.replace(part, num2words(part, lang="ml"))

        return text

#### Tests

In [ ]:
normalizer = MalayalamNormalizer()

In [ ]:
TEST_RESULT = "എന്റെ കമ്പ്യൂട്ടറിന് എന്റെ ഭാഷ"
text_result = normalizer("എന്റെ കമ്പ്യൂട്ടറിനു് എന്റെ ഭാഷ")

assert text_result == TEST_RESULT

In [ ]:
TESTCASE_RESULT = "യുപിഎ ഭരണകാലത്തെ സാമ്പത്തിക വീഴ്ച; ധവളപത്രം ഇറക്കാൻ കേന്ദ്രസർക്കാർ.\n\nയുപിഎ സർക്കാരിന്റെ കാലത്തെ ധനവിനിയോഗത്തിലെ വീഴ്ചകൾ വ്യക്തമാക്കുന്ന ധവളപത്രം ഇറക്കാൻ കേന്ദ്രസർക്കാർ തീരുമാനം. ബജറ്റ് സമ്മേളനം ഇതിനായി ഒരു ദിവസം കൂടി നീട്ടും. വിഹിതങ്ങൾ എപ്രകാരം തെറ്റായി വിനിയോഗിക്കപ്പെട്ടു എന്നതുൾപ്പെടെയുള്ള കാര്യങ്ങൾ വിശദീകരിക്കാനാണ് കേന്ദ്രസർക്കാർ നീക്കം.\n\n"
text_result = normalizer(
    """യുപിഎ ഭരണകാലത്തെ സാമ്പത്തിക വീഴ്ച; ധവളപത്രം ഇറക്കാൻ കേന്ദ്രസർക്കാർ.

യുപിഎ സർക്കാരിന്റെ കാലത്തെ ധനവിനിയോഗത്തിലെ വീഴ്ചകൾ വ്യക്തമാക്കുന്ന ധവളപത്രം ഇറക്കാൻ കേന്ദ്രസർക്കാർ തീരുമാനം. ബജറ്റ് സമ്മേളനം ഇതിനായി ഒരു ദിവസം കൂടി നീട്ടും. വിഹിതങ്ങൾ എപ്രകാരം തെറ്റായി വിനിയോഗിക്കപ്പെട്ടു എന്നതുൾപ്പെടെയുള്ള കാര്യങ്ങൾ വിശദീകരിക്കാനാണ് കേന്ദ്രസർക്കാർ നീക്കം.

"""
)
text_result

'യുപിഎ ഭരണകാലത്തെ സാമ്പത്തിക വീഴ്ച; ധവളപത്രം ഇറക്കാൻ കേന്ദ്രസർക്കാർ.\n\nയുപിഎ സർക്കാരിന്റെ കാലത്തെ ധനവിനിയോഗത്തിലെ വീഴ്ചകൾ വ്യക്തമാക്കുന്ന ധവളപത്രം ഇറക്കാൻ കേന്ദ്രസർക്കാർ തീരുമാനം. ബജറ്റ് സമ്മേളനം ഇതിനായി ഒരു ദിവസം കൂടി നീട്ടും. വിഹിതങ്ങൾ എപ്രകാരം തെറ്റായി വിനിയോഗിക്കപ്പെട്ടു എന്നതുൾപ്പെടെയുള്ള കാര്യങ്ങൾ വിശദീകരിക്കാനാണ് കേന്ദ്രസർക്കാർ നീക്കം.\n\n'

In [ ]:
assert text_result == TESTCASE_RESULT

In [ ]:
normalizer = MalayalamNormalizer(tts_mode=True)
malaylam_text = "ഇന്ത്യയുടെ ജിഡിപി 1.5 ട്രില്യൺ യുഎസ് ഡോളറാണ്."
TESTCASE_RESULT = "ഇന്ത്യയുടെ ജിഡിപി ഒന്ന് പോയിന്റ് അഞ്ച് ട്രില്യൺ യുഎസ് ഡോളറാണ്."
text_result = normalizer(malaylam_text)
text_result

'ഇന്ത്യയുടെ ജിഡിപി ഒന്ന് പോയിന്റ് അഞ്ച് ട്രില്യൺ യുഎസ് ഡോളറാണ്.'

In [ ]:
assert text_result == TESTCASE_RESULT

In [ ]:
TESTCASE_RESULT = "ദുഃഖം"
text_result = normalizer("ദു:ഖം")
text_result

'ദുഃഖം'

In [ ]:
assert text_result == TESTCASE_RESULT

In [ ]:
TESTCASE_RESULT = "എന്റെ"
text_result = normalizer("എൻറെ")
text_result

# Still fails

'എൻറെ'

In [ ]:
# TESTCASE_RESULT  = "കാണ്മാനില്ല"
# text_result = normalizer("കാണ്മാനില്ല")
# text_result

# Still fails

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

<unknown>:19: SyntaxWarning: invalid escape sequence '\X'
<unknown>:19: SyntaxWarning: invalid escape sequence '\X'
<unknown>:19: SyntaxWarning: invalid escape sequence '\X'
<unknown>:19: SyntaxWarning: invalid escape sequence '\X'
